# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [2]:
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-8qdsgial
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-8qdsgial
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 9e8c38aa5b92bbf0e20f65fc611fd43b43196859
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-jmr7s2a3/elegantrl_3f44e38e622e41afab3902f3ead2231b
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-jmr7s2a3/elegantrl_3f44e38e622e41afab3902f3ead2231b
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 37aac1f592e1add9f9fd37ae8db1094656009b76
  Preparing metadata (setup.py) ... done


In [3]:
import pandas as pd
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS
from finrl.config import *
import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [4]:
TRAIN_START_DATE = '2020-01-01'
TRADE_END_DATE = '2020-01-31'
aapl_df_yf = yf.download(tickers = "aapl", start=TRAIN_START_DATE, end=TRADE_END_DATE)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [5]:
aapl_df_yf.head()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-01-02,72.538528,72.598907,71.292319,71.545905,135480400
2020-01-03,71.833298,72.594063,71.608692,71.765674,146322800
2020-01-06,72.405693,72.444336,70.703027,70.954203,118387200
2020-01-07,72.065163,72.671356,71.845385,72.415353,108872000
2020-01-08,73.224403,73.526295,71.768079,71.768079,132079200


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [6]:
aapl_df_finrl = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRAIN_END_DATE,
                                ticker_list = ['aapl']).fetch_data()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (146, 8)


In [7]:
aapl_df_finrl.head()

Price,date,close,high,low,open,volume,tic,day
0,2020-01-02,72.538498,72.598876,71.292289,71.545875,135480400,aapl,3
1,2020-01-03,71.833282,72.594048,71.608677,71.765659,146322800,aapl,4
2,2020-01-06,72.405678,72.444321,70.703012,70.954188,118387200,aapl,0
3,2020-01-07,72.065155,72.671348,71.845377,72.415345,108872000,aapl,1
4,2020-01-08,73.224419,73.526310,71.768094,71.768094,132079200,aapl,2


## Data for the chosen tickers

In [8]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [9]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2022-01-01'
VALID_START_DATE = '2022-01-01'
VALID_END_DATE = '2022-12-31'
TRADE_START_DATE = '2023-01-01'
TRADE_END_DATE = '2024-12-31'

In [10]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (118181, 8)


In [11]:
df_dji = YahooDownloader(
    start_date=TRADE_START_DATE, end_date=TRADE_END_DATE, ticker_list=["^DJI"]
).fetch_data()
df_dji

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (501, 8)


Price,date,close,high,low,open,volume,tic,day
0,2023-01-03,33136.371094,33387.519531,32850.570312,33148.898438,355930000,^DJI,1
1,2023-01-04,33269.769531,33409.101562,33033.480469,33165.140625,379400000,^DJI,2
2,2023-01-05,32930.078125,33191.718750,32812.328125,33191.718750,337920000,^DJI,3
3,2023-01-06,33630.609375,33710.660156,32997.390625,33055.300781,360740000,^DJI,4
4,2023-01-09,33517.648438,33935.109375,33487.660156,33664.390625,327840000,^DJI,0
...,...,...,...,...,...,...,...,...
496,2024-12-23,42906.949219,42957.789062,42516.871094,42800.488281,449390000,^DJI,0
497,2024-12-24,43297.031250,43297.648438,42871.761719,42916.480469,230410000,^DJI,1
498,2024-12-26,43325.800781,43373.980469,43115.089844,43201.851562,270350000,^DJI,3
499,2024-12-27,42992.210938,43238.851562,42761.558594,43142.371094,376960000,^DJI,4


In [12]:
df_raw.head()

Price,date,close,high,low,open,volume,tic,day
0,2009-01-02,2.724325,2.733032,2.556514,2.578128,746015200,AAPL,4
1,2009-01-02,40.463211,40.524945,39.612653,40.188837,6547900,AMGN,4
2,2009-01-02,14.891702,15.038077,14.175236,14.306203,10955700,AXP,4
3,2009-01-02,33.941086,34.173611,32.088389,32.103391,7010200,BA,4
4,2009-01-02,30.233915,30.279031,28.815995,28.944897,7117200,CAT,4


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [13]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (4024, 8)
Successfully added vix
Successfully added turbulence index


In [14]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [15]:
processed_full.head()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,2.724325,2.733032,2.556514,2.578128,746015200.0,4.0,0.0,2.944416,2.619212,100.0,66.666667,100.0,2.724325,2.724325,39.189999,0.0
1,2009-01-02,AMGN,40.463211,40.524945,39.612653,40.188837,6547900.0,4.0,0.0,2.944416,2.619212,100.0,66.666667,100.0,40.463211,40.463211,39.189999,0.0
2,2009-01-02,AXP,14.891702,15.038077,14.175236,14.306203,10955700.0,4.0,0.0,2.944416,2.619212,100.0,66.666667,100.0,14.891702,14.891702,39.189999,0.0
3,2009-01-02,BA,33.941086,34.173611,32.088389,32.103391,7010200.0,4.0,0.0,2.944416,2.619212,100.0,66.666667,100.0,33.941086,33.941086,39.189999,0.0
4,2009-01-02,CAT,30.233915,30.279031,28.815995,28.944897,7117200.0,4.0,0.0,2.944416,2.619212,100.0,66.666667,100.0,30.233915,30.233915,39.189999,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [16]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
valid = data_split(processed_full, VALID_START_DATE,VALID_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(train))
print(len(trade))

94917
94917
14500


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [17]:
import os
os.makedirs('./data', exist_ok=True)
train.to_csv('./data/train_data.csv')
valid.to_csv('./data/valid_data.csv')
trade.to_csv('./data/trade_data.csv')